# Multicut

Use the `elf.segmentation` module for boundary based multicut segmentation: [Multicut brings automated neurite segmentation closer to human performance](https://www.nature.com/articles/nmeth.4151). We use data from the [ISBI 2012 EM Segmentation challenge](http://brainiac2.mit.edu/isbi_challenge/home) and [The Mutex Watershed: Efficent, Parameter-Free Image Partitionong](http://openaccess.thecvf.com/content_ECCV_2018/html/Steffen_Wolf_The_Mutex_Watershed_ECCV_2018_paper.html).
You can obtain the example data [here](https://hcicloud.iwr.uni-heidelberg.de/index.php/s/6LuE7nxBN3EFRtL).

The boundary based segmentation approach works as follows:
1. Predict pixel-wise affinity (or boundary) map. Here, we use pre-computed results for this step.
2. Compute a watershed over-segmentation based on the affinity maps.
3. Compute the region adjacency graph defined by the watershed over-segmentation.
4. Compute weights for the edges of this graph by accumulating the affinity (or boundary) map over the edge pixels.
5. Partition the graph based on the edge weights via Multicut and project the result back to the pixel level.

## Preparation

In [1]:
%gui qt5 
import numpy as np
import imageio

# import napari for data visualisation
import napari

# import the segmentation functionality from elf
import elf.segmentation.multicut as mc
import elf.segmentation.features as feats
import elf.segmentation.watershed as ws
from elf.segmentation.utils import seg_to_edges

# import the open_file function from elf, which supports opening files
# in hdf5, zarr, n5 or knossos file format
from elf.io import open_file

/g/kreshuk/pape/Work/software/src/napari/napari/__init__.py:27: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.6. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [2]:
# read the data
# you can download the example data from here:
# https://hcicloud.iwr.uni-heidelberg.de/index.php/s/6LuE7nxBN3EFRtL
data_path = '/home/pape/Work/data/isbi/isbi_test_volume.h5'  # adjust this path
with open_file(data_path, 'r') as f:
    # load the raw data
    raw = f['raw'][:]
pmap_path = '/home/pape/Work/data/isbi/fabian/isbi_test_prob.tif'
pmaps = 1. - np.asarray(imageio.volread(pmap_path))

In [3]:
print(pmaps.min(), pmaps.max())

0.0 0.9999947


In [8]:
# visualize the raw data and the affinity maps with napari
# TODO switch to new napari syntax
# napari.view_image(raw, name='raw')
# napari.view_image(affs, name='affinities')
viewer = napari.Viewer()
viewer.add_image(raw, name='raw')
viewer.add_image(pmaps, name='probabilities')

<Image layer 'probabilities' at 0x7fa45f3a2978>

## Set up the Multicut problem

In [4]:
# compute watershed over-segmentation based on the probability maps


# next, we run the distance transform watershed.
# the data is very anisotropic, so we apply the watershed in 2d
# and stack the watershed results along z, with the appropriate id offset
watershed = np.zeros_like(raw, dtype='uint64')
offset = 0
for z in range(watershed.shape[0]):
    # the threshold parameter determines at which value the input map is thresholded before applying
    # the distance transform.
    # the parameter sigma_seeds determines how strong the seed map is smoothed before seeds are
    # computed via local minima. This controls the degree of over-segmentation
    wsz, max_id = ws.distance_transform_watershed(pmaps[z], threshold=.5, sigma_seeds=2.)
    wsz += offset
    offset += max_id
    watershed[z] = wsz

In [5]:
# visualize the watershed result
# TODO switch to new napari syntax
# napari.view_image(raw, name='raw')
# napari.add_labels(watershed, name='watershed')
viewer = napari.Viewer()
viewer.add_image(raw, name='raw')
viewer.add_labels(watershed, name='watershed')

<Labels layer 'watershed' at 0x7fd670388a90>

In [6]:
# compute the region adjacency graph
rag = feats.compute_rag(watershed)

# compute the edge costs
costs = feats.compute_boundary_features(rag, pmaps)[:, 0]

# transform the edge costs from [0, 1] to  [-inf, inf], which is
# necessary for the multicut. This is done by intepreting the values
# as probabilities for an edge being 'true' and then taking the negative log-likelihood.

# in addition, we weight the costs by the size of the corresponding edge
# for z and xy edges
z_edges = feats.compute_z_edge_mask(rag, watershed)
xy_edges = np.logical_not(z_edges)
edge_populations = [z_edges, xy_edges]
edge_sizes = feats.compute_boundary_mean_and_length(rag, pmaps)[:, 1]
costs = mc.transform_probabilities_to_costs(costs, edge_sizes=edge_sizes,
                                            edge_populations=edge_populations)

77466.0
-0.028835723 3.0622032
494.0
-4.4614334 3.3327825


In [14]:
print(costs.min())
print(costs.max())

-4.4614334
3.3327825


## Solve the Multicut

In [12]:
# run the multicut partitioning, here, we use the kernighan lin 
# heuristics to solve the problem, introduced in
# http://xilinx.asia/_hdl/4/eda.ee.ucla.edu/EE201A-04Spring/kl.pdf
node_labels = mc.multicut_kernighan_lin(rag, costs)

# map the results back to pixels to obtain the final segmentation
segmentation = feats.project_node_labels_to_pixels(rag, node_labels)
edges = 1. - seg_to_edges(segmentation, True)

In [13]:
# visualize the segmentation result
# TODO switch to new napari syntax
# napari.view_image(raw, name='raw')
# napari.add_labels(segmentation, name='multicut-segmentation')
viewer = napari.Viewer()
viewer.add_image(raw, name='raw')
viewer.add_image(pmaps, name='boundaries')
viewer.add_labels(segmentation, name='multicut')
viewer.add_image(edges.astype('float32'), name='edge_volume')

<Image layer 'edge_volume' at 0x7fd668912fd0>

In [14]:
imageio.volwrite('mc-result-segmentation.tif', segmentation)
imageio.volwrite('mc-result-edges.tif', edges.astype('float32'))